# Exploring the transformation functions used in the code

March 26, 2020

May 19, 2020 : Added linear+log transformation



In [1]:
import numpy as np
import matplotlib.pyplot as plt
# import pandas as pd

import subprocess as sp
import sys
import os
import glob

from scipy import fftpack

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/')
from modules_image_analysis import *

[NbConvertApp] WARNING | pattern 'modules_image_analysis.ipynb' matched no files
This application is used to convert notebook files (*.ipynb) to various other
formats.


Options
-------

Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.

--debug
    set log level to logging.DEBUG (maximize logging output)
--generate-config
    generate default config file
-y
    Answer yes to any questions instead of prompting.
--execute
    Execute the notebook prior to export.
--allow-errors
    Continue notebook execution even if one of the cells throws an error and include the error message in the cell output (the default behaviour is to abort conversion). This flag is only relevant if '--execute' was specified, too.
--stdin
    read a single notebook file from stdin. Write the resulting notebook with default basename 'notebook.*'
--stdout
    Write notebook outp

## Extract image data 

In [5]:
data_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/peter_dataset/'
fname='raw_val.npy'
samples = np.load(data_dir+fname, allow_pickle=True)
samples = samples.transpose(0,3,1,2)[:,0,:,:]
### Re-convert image values to those before re-scaling
# samples=f_invtransform(samples)
samples.shape

(2680, 128, 128)

## Transformation functions

### Pixel intensity histogram

In [6]:
hist=f_pixel_intensity(samples[:100])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)

## Transform
plt.figure()
x=np.array([0,1,10,30,50,51,55,60,100,500,1000,5000,10000,15000],dtype=float)
arr=f_transform(x)
plt.plot(x,arr,linestyle='',marker='o')
plt.title('Transform')

## Inverse transform
x=arr
arr_inv=f_invtransform(arr)

plt.figure()
plt.plot(x,arr_inv,linestyle='',marker='o')
plt.title('Inv-transform')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Inv-transform')

In [11]:
# ###pixel intensities after transformation
# arr1=f_transform(samples[:100])
# hist=f_pixel_intensity(arr1)

# ###pixel intensities after inverse transformation
# arr1=f_transform(samples[:100])
# hist=f_pixel_intensity(arr1)


In [14]:
### Comparing array with f_invtransform(f_transform)
# f_pixel_intensity(samples[:100])
arr=f_invtransform(f_transform(samples[:100]))
f_compare_pixel_intensity([samples[:100],arr])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Spectrum

In [15]:
arr=f_invtransform(f_transform(samples[:100]))
f_compare_spectrum([samples[:100],arr])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## New transformations

#### Log-linear transformations

In [17]:
def f_1(x):
    y=0.03*x-1.0
    return y

def f_2(x):
    a=0.5/(np.log(15000)-np.log(51))
    b=0.5-a*np.log(51)
#     print(a,b)
    y=a*np.log(x)+b
    return y


def f_3(x):
    a=(15e3-np.exp(0.5)*50)/(np.exp(0.5)-1)
    b=0.5-np.log(50+a)
#     print(a,b)
    y=np.log(x+a)+b
    return y


In [18]:
for x in [51,55,60,100,500,1000,5000,10000,15000]:
    print(x,f_2(x),f_3(x))
    
x=[51,55,60,100,500,1000,5000,10000,15000]
plt.figure()
plt.plot(x,f_2(x),marker='*',color='k',label='alog(x)+b')
plt.plot(x,f_3(x),marker='s',label='log(x+a)+b')

plt.legend()

51 0.5 0.5000433917857006
55 0.5066421375986419 0.5002169401024812
60 0.5142962267515128 0.5004338331521598
100 0.5592317857686026 0.5021672860947586
500 0.7008084587736643 0.5193385266884558
1000 0.7617822130069665 0.5403960709767652
5000 0.9033588860120283 0.6945745145401201
10000 0.9643326402453305 0.8589028049226037
15000 1.0 1.0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Picking best one

In [19]:
x=np.linspace(0,50,num=500)
y=f_1(x)

# plt.figure()
# plt.plot(x,y,marker='o')

In [20]:
def f_transform_new(x):
    if x<=50:
        a=0.03; b=-1.0
        return a*x+b
    elif x>50: 
        a=0.5/np.log(300)
        b=0.5-a*np.log(50)
        return a*np.log(x)+b

def f_invtransform_new(y):
    if y<=0.5:
        a=0.03;b=-1.0
        return (y-b)/a
    elif y>0.5: 
        a=0.5/np.log(300)
        b=0.5-a*np.log(50)
        return np.exp((y-b)/a)


In [28]:
# a1=np.load('/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/peter_dataset/pre_norm_train.npy').astype('float')
# a2=np.load('/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/peter_dataset/old_pre_norm_train.npy')
# print(a1.shape,a2.shape)

In [22]:
print(type(a1),type(a2))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [23]:
x_1=[0,1,10,30,50,51,55,60,100,500,1000,5000,10000,15000]
y_1=np.vectorize(f_transform_new)(x_1)
x_2=np.vectorize(f_invtransform_new)(y_1)
for x,y,x2 in zip(x_1,y_1,x_2): print(x,y,x2)
plt.figure()
plt.plot(x_1,y_1,linestyle='',marker='*')

0 -1.0 0.0
1 -0.97 1.0000000000000009
10 -0.7 10.000000000000002
30 -0.10000000000000009 29.999999999999996
50 0.5 50.0
51 0.5017359206267218 51.0
55 0.5083549977780377 55.000000000000064
60 0.5159825131484299 59.99999999999993
100 0.5607620630379777 99.99999999999996
500 0.7018472043091751 499.99999999999983
1000 0.7626092673471528 999.9999999999998
5000 0.9036944086183504 5000.000000000004
10000 0.9644564716563281 10000.00000000001
15000 1.0 15000.000000000004


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### optimizing memory usage of new transformation

In [40]:
import tensorflow.keras.backend as K

/global/homes/v/vpa/.conda/envs/v_py3/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/global/homes/v/vpa/.conda/envs/v_py3/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/global/homes/v/vpa/.conda/envs/v_py3/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/global/homes/v/

In [41]:
size=4
a1=np.ones(np.int(size**2/2))
x=np.vstack([a1,100*a1]).reshape(size,size)
x

array([[  1.,   1.,   1.,   1.],
       [  1.,   1.,   1.,   1.],
       [100., 100., 100., 100.],
       [100., 100., 100., 100.]])

In [42]:
def f_transform_new(x):
    if x<=50:
        a=0.03; b=-1.0
        return a*x+b
    elif x>50: 
        a=0.5/np.log(300)
        b=0.5-a*np.log(50)
        return a*np.log(x)+b

def f_invtransform_new(y):
    if y<=0.5:
        a=0.03;b=-1.0
        return (y-b)/a
    elif y>0.5: 
        a=0.5/np.log(300)
        b=0.5-a*np.log(50)
        return np.exp((y-b)/a)
    
    
x,np.vectorize(f_transform_new)(x)

(array([[  1.,   1.,   1.,   1.],
        [  1.,   1.,   1.,   1.],
        [100., 100., 100., 100.],
        [100., 100., 100., 100.]]),
 array([[-0.97      , -0.97      , -0.97      , -0.97      ],
        [-0.97      , -0.97      , -0.97      , -0.97      ],
        [ 0.56076206,  0.56076206,  0.56076206,  0.56076206],
        [ 0.56076206,  0.56076206,  0.56076206,  0.56076206]]))

In [53]:
lesser_idx=np.where(x<50)
lesser_idx

(array([0, 0, 0, 0, 1, 1, 1, 1]), array([0, 1, 2, 3, 0, 1, 2, 3]))

In [61]:
from tensorflow.keras import backend as K


In [63]:
K.where


AttributeError: module 'tensorflow.keras.backend' has no attribute 'where'

In [39]:
greater_mask=x>50
lesser_mask=np.logical_not(greater_mask)
print(lesser_mask)
# print(greater_mask)

# x=x[lesser_mask]
# a=0.03; b=-1.0
# y=a*x+b
# print(y)

x=x[greater_mask]
a=0.5/np.log(300)
b=0.5-a*np.log(50)
y=a*np.log(x)+b
print(y)

[False False False False False False False False]
[0.56076206 0.56076206 0.56076206 0.56076206 0.56076206 0.56076206
 0.56076206 0.56076206]


In [80]:
# x=np.arange(10).reshape(5,2)
# import tensorflow as tf
# import tensorflow.keras.backend as K
# value = 5
# wh = K.where(K.equal(x,value))

In [65]:
K.equal(x,value)

<tf.Tensor: id=5, shape=(5, 2), dtype=bool, numpy=
array([[False, False],
       [False, False],
       [False,  True],
       [False, False],
       [False, False]])>